# Californa API Analysis Examples

This IPython notebook is an example of using IPython and Ambry for analyzing Academic Performance Index scores for California, from 2000 to 2012. This data was extracted from the California Department of Education website and loaded into Ambry to create a single dataset that combines multiple years of data and harmonizes column names across the years. [An earlier version of this set](http://data.sandiegodata.org/dataset/cde_ca_gov-api-combined-429e-extract) can be downloaded from the San Diego Regional Data Library Data Repository, which also hosts the [documentation for the dataset](http://data.sandiegodata.org/storage/f/2013-11-05T211612/documentation.html). 

In this example, we will show how to get datasets from an Ambry library and perform some analysis using Pandas. 




## Getting Data With Ambry

Ambry is a data library and package management system that makes it much easier for analysts to get data to use in their analysis. In these first cells, we will get a library objectand look for the API dataset. 

In this case, Ambry has been previously configured to connect to a remote Ambry library that hosts the dataset. 

First we will construct the interactive library using ilibrary() and search for files that have 'api' in the name. 


In [1]:
import ambry
import pandas as pd
l = ambry.ilibrary()
l.find(name='api')

LS,d008002,cde.ca.gov-api-orig-0.1.2
LS,d007004,cde.ca.gov-api-combined-0.1.4
L,d007002,cde.ca.gov-api-combined-0.1.2
L,d007003,cde.ca.gov-api-combined-0.1.3
L,d007001,cde.ca.gov-api-combined-0.1.1
L,d008001,cde.ca.gov-api-orig-0.1.1


The last line of the input cell, `l.find(name='api')` returns a list of records in a way that allows IPython to print it in HTML form. This is a frequently-used way to get more information about objects with using IPython Notebook. 

The find command looks for all of the records of datasets in the local library and displays the ones that have 'api' in the name. The first column of the output tells us where the data lives. 'SLR' means that the data is on the local machine as a Source package, in the Local librayr, and also in the Remote library. The second column is the unique, versioned id number. The last two items are the same dataset, but one is a later version than the other. 

Each of these datasets may be composed of multiple partitions, with each partition holding a part of a dataset, broken up by time, geographic location, or other factor. Now we will fetch one of the packages and look at what partitions are available. 


In [2]:
b =  l.get('cde.ca.gov-api-combined-api_base')
b


Identity,
Vid,d007004
Sname,cde.ca.gov-api-combined
Vname,cde.ca.gov-api-combined-0.1.4
Locations,L
Source,
Db,/data/library/cde.ca.gov/api-combined-0.1.4.db
Created,
Built,2014-04-06T17:13:57.724470
Build_Time,268.45s
Prepared,2014-04-06T17:09:29.127678


When we asked for the dataset without a version number, Ambry selected the latest version and returned it. This object is also know as a "Bundle" since it provides the grouping for multiple partitions. On the last line we can see that this bundle has two partitions. 

In [3]:
b.partitions

Id,Name,table
p007001,cde.ca.gov-api-combined-api_base,api_base
p007002,cde.ca.gov-api-combined-api_growth,api_growth


One partition in the bundle is for the base API scores, and the other is for year-to-year growth in the scores. Let's get the base partition and examine it. If you print out the `p` object, you'll get information about the partition, but we'll focus on the main table in the partition. 

In [4]:
p = l.get('cde.ca.gov-api-combined-api_base').partition
assert(p.identity.on.revision >= 2)
p.table


Displaying the table presents information about the table and the whole table schema. Now we can use that schema to construct a query, load the data into a Pandas data frame and do some analysis. 

## Analyzing Data

In [5]:
q = """
SELECT 
cds, year, api, valid, 
aa_num, pct_aa,  aa_api, 
wh_num,  pct_wh, wh_api,
hi_num,  pct_hi, hi_api,
as_num,  pct_as, as_api,
el
FROM api_base 
---WHERE wh_sig = 'Yes' AND aa_sig = 'Yes'
"""

df = p.select(q).pandas
df.head()

,cds,year,api,valid,aa_num,pct_aa,aa_api,wh_num,pct_wh,wh_api,hi_num,pct_hi,hi_api,as_num,pct_as,as_api,el
0,1611190130229,2000,731,1090,39,4,NaN,414,38,760,96,9,NaN,414,38,749,16
1,1611190132878,2000,622,840,160,23,528,191,23,734,92,11,NaN,195,21,642,18
2,1611196000004,2000,622,472,138,31,555,90,19,714,62,13,NaN,75,15,672,25
3,1611196090005,2000,774,272,16,7,NaN,61,24,781,21,7,NaN,104,38,813,26
4,1611196090013,2000,811,216,6,3,NaN,128,59,851,19,9,NaN,17,7,NaN,9


Ambry datasets are stored in Sqlite, a complete SQL database system, so you can run standard SQL queries to extract data. If you don't want to use SQL, you can omit the query to get the whole dataset into Pandas, but this can consume a lot of memory. 

Now that we've extracted some data, we'll convert the Pandas Dataframe into a Time-series panel so we can analyze school performance across multiple years. 

In [6]:
mldf = df.set_index(['cds','year'])
pan = mldf.to_panel() # Convert to a time-series panel
df.head()

,cds,year,api,valid,aa_num,pct_aa,aa_api,wh_num,pct_wh,wh_api,hi_num,pct_hi,hi_api,as_num,pct_as,as_api,el
0,1611190130229,2000,731,1090,39,4,NaN,414,38,760,96,9,NaN,414,38,749,16
1,1611190132878,2000,622,840,160,23,528,191,23,734,92,11,NaN,195,21,642,18
2,1611196000004,2000,622,472,138,31,555,90,19,714,62,13,NaN,75,15,672,25
3,1611196090005,2000,774,272,16,7,NaN,61,24,781,21,7,NaN,104,38,813,26
4,1611196090013,2000,811,216,6,3,NaN,128,59,851,19,9,NaN,17,7,NaN,9


Using Pandas for data analysis is a bit beyond the scope of this introduction, and panels can be complicated. FOr this tutorial, is is most important to know that a panel works like a 3-dimentional structure, where two of the dimensions are CDS -- the school identification number -- and the year. The third is the data columns. Panels provide a convient way to analyze time series. 

In particular, with the object `pan`, we can extract a single column. For instance `pan['pct_wh']` extracts a 2-dimensional table of percentages of White test takers, with one entry for each school and each year. Then `pan['pct_wh'].mean()` will compute the mean value for each year across all of the schools. ( If we had set the indices the other way, with `df.set_index(['year','cds'])` then `mean()` would calculate the mean value for each school across the years. 

Finally, we will perform this calculation for the four largest racial categories and combine them into a single table. 


In [7]:
racial_pcts = pd.concat([pan['pct_wh'].mean(), pan['pct_aa'].mean(),pan['pct_hi'].mean(), pan['pct_as'].mean()], axis=1)
racial_pcts.columns = ['pct_wh', 'pct_aa', 'pct_hi', 'pct_as']
racial_pcts

,pct_wh,pct_aa,pct_hi,pct_as
year,,,,
2000,41.583866,8.068314,37.180228,7.684532
2001,40.582148,7.956793,38.696600,7.823513
2002,39.264540,7.874262,39.876774,7.470670
2003,38.175535,7.853156,40.768910,7.799948
2004,36.548480,8.058513,42.218551,7.542833
2005,37.419843,7.689846,41.728833,6.885074
2006,36.110113,7.740394,43.053814,6.883961
2007,34.854257,7.597130,44.374363,6.861903
2008,34.124119,7.575547,44.982481,6.921672


Now, for a plot of the numbers. 


# Creating Plots

Now that we have a table of data, the next thing to do is plot it. 


In [8]:
plot(racial_pcts)

NameError: name 'plot' is not defined

That was really easy, but not pretty. Getting a proper legend takes a bit more work.

In [ ]:
fig = figure()
ax = subplot(111)

for col in list(racial_pcts.columns.values):
    ax.plot(racial_pcts[col], label=col)
    
ax.legend(loc='best', bbox_to_anchor=(1, 0.5))

show()


Now we can do the same analysis with the racial decomposition of the API scores. 

In [ ]:
racial_api = pd.concat([pan['wh_api'].mean(), pan['aa_api'].mean(),pan['hi_api'].mean(), pan['as_api'].mean(),  pan['api'].mean()], axis=1)
racial_api.columns = ['wh_api', 'aa_api', 'hi_api', 'as_api', 'api']
racial_api

In [ ]:
fig = figure()
ax = subplot(111)

for col in list(racial_api.columns.values):
    ax.plot(racial_api[col], label=col)
    
ax.legend(loc='best', bbox_to_anchor=(1, 0.5))

show()

## More Complex Analysis

In [ ]:
# We need to run the query again to add the significance criteria. 
q = """
SELECT 
cds, year, api, valid, 
aa_num, pct_aa,  aa_api, 
wh_num,  pct_wh, wh_api,
hi_num,  pct_hi, hi_api,
as_num,  pct_as, as_api,
el
FROM api_base 
WHERE wh_sig = 'Yes' AND aa_sig = 'Yes'
"""

df = p.select(q).pandas
df.head()

x = df.wh_api
y = df.aa_api
fit = polyfit(x,y,1)
fit_fn = poly1d(fit) # fit_fn is now a function which takes in x and returns an estimate for y

plot(x,y, 'b,', x, fit_fn(x), '--k', x,x, '--r')

In [ ]:
aa_better = df[df.aa_api > df.wh_api]
aa_better.count()